**Навигация по уроку**
1. [Знакомство с контурными моделями](https://colab.research.google.com/drive/1TzGgfcUDo-CFg_RbU7dqAQuoSpR-QrjR)
2. [Кто такая LoRa и как обучать LLM?](https://colab.research.google.com/drive/1HlMzFKtaFffasPcNvZYNcpv8nsZ-FZt_)
3. [Пример обучения с LoRA](https://colab.research.google.com/drive/1I3kU-TO7qCrfxidpoW1vQS0AYTSd9oxb)
4. Домашняя работа

В домашней работе вам необходимо получить навыки работы с большими языковыми моделями. Для этого необходимо:

1. Создать учетную запись на Hugging Face (HF).
2. Получить токен для работы с HF.
3. Загрузить адаптер русско-язычной [Saiga](https://huggingface.co/IlyaGusev/saiga_mistral_7b_lora)
4. Для получению 3 баллов вам достаточно сгенерировать 3 шутки с помощью Saiga.
5. Для 4-х баллов выполните пункт 4, используя квантованную версию модели [saiga_mistral_7b_gguf](https://huggingface.co/IlyaGusev/saiga_mistral_7b_gguf).
6. Хотите 5 баллов? Используя датасет [ru_turbo_saiga](https://huggingface.co/datasets/IlyaGusev/ru_turbo_saiga), на котором обучалась данная модель произведите оценку точности, как мы это делали в практической части урока. Хотя на данном датасете оценивать модель не совсем корректно, так как она уже "видела" эти данные, но поняв принцип, вы всегда сможете повторить процедуру и на новых данных. Также вы можете оценить модель и другими способами, например, изложенными в [статье](https://www.philschmid.de/evaluate-llm).

# Авторизация на HF

In [ ]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

# Вставьте ваш токен (здесь указан временный токен)
login(hf_token)

# Загрузка квантованной модели saiga_mistral_7b_gguf

Для экспериментов взял модель model-q2_K.gguf, которая весит всего 3 Гб. Она подойдет как экспериментальная даже с недостаточной точностью, т.к. цель состоит в загрузке и получении от нее генерации как процесса.

In [ ]:
!pip install -q ctransformers[cuda]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 59.8 MB/s eta 0:00:0000:0100:01


In [ ]:
!pip install 'transformers<4.50'

In [ ]:
import transformers

In [ ]:
print(transformers.__version__)

4.49.0


In [ ]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers.pipelines import pipeline

2025-06-24 18:43:42.534877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750790622.949996      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750790623.067439      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-v0.1", use_fast=True
)

# Устанавливаем кастомный чат-шаблон для формата Saiga/Mistral
chat_template = """{% for message in messages %}
    {% if message['role'] == 'system' %}
        {{ '<|system|>\n' + message['content'] }}
    {% elif message['role'] == 'user' %}
        {{ '<|user|>\n' + message['content'] + '<|assistant|>\n' }}
    {% elif message['role'] == 'assistant' %}
        {{ message['content'] + eos_token }}
    {% endif %}
{% endfor %}"""
tokenizer.chat_template = chat_template

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Загружаем LLM
# Используйте `gpu_layers`, чтобы указать, сколько слоев будет загружено в графический процессор.
model = AutoModelForCausalLM.from_pretrained(
    "IlyaGusev/saiga_mistral_7b_gguf",          # загружаем Зефирку в GGUF формате
    model_file="model-q2_K.gguf",               # выбираем конкретный файл модели с 4-х битным квантованием
    model_type="mistral",                       # уточняем тип модели (на базе Mistral AI)
    gpu_layers=50,                              # сколько слоев загружать в GPU
    hf=True,                                    # создавать модель-трансформер с помощью HuggingFace.
)

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model-q2_K.gguf:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

ggml_cuda_set_main_device: using device 0 (Tesla T4) as main device
CTransformersModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [ ]:
# Create a pipeline
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device='cuda')

Device set to use cuda


# Получаем ответы от модели

In [ ]:
result = pipe("Почему небо голубое?", max_new_tokens=150)
print(result[0]["generated_text"])

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2105: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


Почему небо голубое?

Вот почему небо голубое:

1. Голубой цвет - это результат смешения двух основных цветов: синего и зеленого.
2. Голубой цвет - это результат смешения двух основных цветов: синего и зеленого.
3. Голубой цвет - это результат смешения двух основных цветов: синего и зеленого.
4. Голубой цвет - это результат смешения двух основных цветов: синего и зеленого.
5. Голубой


In [ ]:
# Попросим модель рассказать нам шутку про HuggingFace
messages = [
    {
        "role": "system",
        "content": "Ты дружелюбный чат-бот.",
    },
    {
        "role": "user",
        "content": "Расскажи несколько шуток."
    },
]

# Мы используем шаблон чата токенизатора для форматирования каждого сообщения
# See https://huggingface.co/docs/transformers/main/en/chat_templating
prompt = pipe.tokenizer.apply_chat_template(
    messages,                                   # Инструкции для генерации промпта
    tokenize=False,                             # Не спешим токенизировать
    add_generation_prompt=True                  # Добавить <|assistant|> - промпт ожидание ответа от ассистента
)

In [ ]:
print("Сформированный промпт:\n", prompt)

Сформированный промпт:
         <|system|>
Ты дружелюбный чат-бот.
        <|user|>
Расскажи несколько шуток.<|assistant|>




In [ ]:
# We will use the same prompt as we did originally
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)
print(outputs[0]["generated_text"])

        <|system|>
Ты дружелюбный чат-бот.
        <|user|>
Расскажи несколько шуток.<|assistant|>

1. В одной деревне жил старичок, который слышал, что у него есть сын.

2. На одной далекой планете живут разные существа, которые постоянно ссорятся, но сразу же примиряются.

3. Сейчас я уже не помню, а скажут ли я эту шутку или нет.

4. Разве нет?

5. Это я проверяю.

6. В одной стране живут разные нации.


In [ ]:
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.8,  # Увеличено с 0.1 для большего разнообразия
    top_p=0.9,
    top_k=40,         # Добавлен параметр top-k
    repetition_penalty=1.2,  # Штраф за повторения
    #num_return_sequences=1   # Количество вариантов ответа
)
print(outputs[0]["generated_text"])

        <|system|>
Ты дружелюбный чат-бот.
        <|user|>
Расскажи несколько шуток.<|assistant|>

* Если бы я был волком, я был бы волчикой!
* В прошлый раз я сказал "вчера", но сегодня у меня есть времени.
* У людей без волос массаж головы - это не только забавное, но и важно для здоровья.
* Мне всегда казалось, что умные боты обитают в облаках. Но на самом деле они там никому не помогут!


## Результат

Довольно странные генерации по натуральным меркам, хотя это можно списать на сильную квантованность -  всего 2 бита, и сложность юмора как такового для LLM. Тем не менее текст получился "почти" на 100% связным, но даже с ошибками он интерпретируем человеком.

# Оценка на датасете ru_turbo_saiga

Оценим модель по методу сравнения с генерацией ответов от доверенной модели. Выберем случайный диалог из датасета и сгенерируем по запросам пользователя ответы нашей модели и сравним с теми, что представлены в датасете (они сгенерированы моделью GPT-3.5-turbo).

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
!pip install jsonlines

In [ ]:
# Загрузить датасет из HF
dataset = load_dataset("IlyaGusev/ru_turbo_saiga", split='train', trust_remote_code=True)

In [ ]:
print(dataset.column_names)  # Список колонок
print(dataset[0])  # Первая запись

['messages', 'seed', 'source', 'model_name']
{'messages': {'role': ['user', 'bot', 'user', 'bot'], 'content': ['Мне нужно найти название фильма. В нем детей из детдомов набирают в группу для спец операций. Ты можешь мне помочь?', 'Конечно. Навскидку я могу предложить несколько фильмов. Но, скорее всего, ты ищешь фильм под названием "Сиротский батальон" (англ. "The Little Bastards"). Эту картина, снятая в 2003 году, рассказывает о группе детей совершающих различные военные миссии, посланные на линию фронта.', 'Да, спасибо. Извини, не было времени на поиск поисковиком. А есть ли другие фильмы на эту тему?', 'Конечно. Есть еще один фильм, который может заинтересовать тебя. Он называется "Солдаты неудачи" (англ. "Toy Soldiers"). В этом фильме подростки из интернатной школы похищаются террористами, и вынуждены сражаться за свою жизнь. Этот фильм был выпущен в 1991 году.']}, 'seed': 'Как называется фильм в котором детей из детдомв набирают в группу для спец операций??', 'source': 'yandex_q',

In [ ]:
dataset['messages'][0]

{'role': ['user', 'bot', 'user', 'bot'],
 'content': ['Мне нужно найти название фильма. В нем детей из детдомов набирают в группу для спец операций. Ты можешь мне помочь?',
  'Конечно. Навскидку я могу предложить несколько фильмов. Но, скорее всего, ты ищешь фильм под названием "Сиротский батальон" (англ. "The Little Bastards"). Эту картина, снятая в 2003 году, рассказывает о группе детей совершающих различные военные миссии, посланные на линию фронта.',
  'Да, спасибо. Извини, не было времени на поиск поисковиком. А есть ли другие фильмы на эту тему?',
  'Конечно. Есть еще один фильм, который может заинтересовать тебя. Он называется "Солдаты неудачи" (англ. "Toy Soldiers"). В этом фильме подростки из интернатной школы похищаются террористами, и вынуждены сражаться за свою жизнь. Этот фильм был выпущен в 1991 году.']}

In [ ]:
import random
# Выбор случайного диалога
random_sample = random.choice(dataset)
messages = random_sample['messages']
seed = random_sample['seed']

# Формирование системного промпта
system_prompt = f"""Идёт диалог между пользователем и ИИ ассистентом.
Пользователь и ассистент общаются на тему: {seed}
Реплики человека начинаются с [Пользователь], реплики ассистента начинаются с [Ассистент].
Пользователь задаёт вопросы на основе темы и предыдущих сообщений.
Пользователь обрывает беседу, когда у него не остается вопросов.
Ассистент даёт максимально полные, информативные, точные и творческие ответы.
Ассистент старается не задавать вопросов, за исключением уточняющих.
Ассистент может отвечать несколькими абзацами.
Ассистент может использовать Markdown.
Закончи диалог точно в таком же формате."""

# Подготовка истории диалога
dialog_history = [system_prompt]
original_responses = []
generated_responses = []

for role, content in zip(messages['role'], messages['content']):
    if role == 'user':
        # Добавляем вопрос пользователя
        user_message = f"[Пользователь] {content}"
        dialog_history.append(user_message)

        # Формируем промпт для генерации
        current_prompt = "\n".join(dialog_history) + "\n[Ассистент]"

        # Генерируем ответ модели
        output = pipe(
            current_prompt,
            max_new_tokens=256,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.1,
            eos_token_id=tokenizer.eos_token_id
        )

        # Извлекаем сгенерированный ответ
        full_output = output[0]['generated_text']
        generated_response = full_output[len(current_prompt):].strip()
        generated_responses.append(generated_response)

        # Добавляем сгенерированный ответ в историю
        dialog_history.append(f"[Ассистент] {generated_response}")
    else:
        # Сохраняем оригинальный ответ ассистента
        original_responses.append(content)
        dialog_history.append(f"[Ассистент] {content}")

# Вывод результатов сравнения
print("="*80)
print(f"ТЕМА ДИАЛОГА: {seed}")
print("="*80)

# Выводим весь диалог с сравнением
for i, (role, content) in enumerate(zip(messages['role'], messages['content'])):
    if role == 'user':
        print(f"\n[Пользователь] {content}")
    else:
        # Индекс текущего ответа ассистента
        resp_idx = len([r for r in messages['role'][:i] if r == 'bot'])

        print("\n" + "-"*40)
        print("ОРИГИНАЛЬНЫЙ ОТВЕТ:")
        print(content)

        if resp_idx < len(generated_responses):
            print("\nСГЕНЕРИРОВАННЫЙ ОТВЕТ:")
            print(generated_responses[resp_idx])
        print("-"*40)

print("\n" + "="*80)
print("СТАТИСТИКА:")
print(f"Всего реплик: {len(messages['role'])}")
print(f"Вопросов пользователя: {len([r for r in messages['role'] if r == 'user'])}")
print(f"Ответов ассистента: {len(original_responses)}")
print(f"Сгенерированных ответов: {len(generated_responses)}")

ТЕМА ДИАЛОГА: Доброго времени суток! У меня есть вопрос по кредиту. Что будет с кредитной историей, если я от него откажусь в течении 14 дней?

[Пользователь] У меня есть вопрос по кредиту. Что будет с кредитной историей, если я от него откажусь в течении 14 дней?

----------------------------------------
ОРИГИНАЛЬНЫЙ ОТВЕТ:
Хороший вопрос! Если вы откажетесь от кредита в течение 14 дней после его оформления, это не повлияет на вашу кредитную историю. Это объясняется тем, что такой короткий период времени не даст кредиторам достаточного времени, чтобы отправить информацию в бюро кредитных историй. Однако, если вы не выполняете обязательства по выплате кредита после этого периода, это может привести к задержкам и негативным записям в вашей кредитной истории.

СГЕНЕРИРОВАННЫЙ ОТВЕТ:
Если вы откажетесь от кредита в течение 14 дней после его получения, ваша кредитная история не будет зарегистрирована как кредит. Вместо этого она будет определена как "неудачное обращение". Это может повлият

## Ошибки

При использовании библиотеки transformers новее 4.50 у модели отключатеся метод generation(), что не позвоволяет получать ответы от нее.

Решение: установил версию старше 4.50, последняя доступная - 4.49

При первой успешной генерации, модель выдавала только 1 вариант ответа несколько раз. Помогло уточнение промпта - указать множественное число в запросе, так же добавил штраф за повторения в последней версии output.

При попытке скачать оценочный датасет, код выдал запрос на выполнение внутреннего скрипта и доверие к нему, можно установить доверие по умолчанию 'trust_remote_code=True'

Потребовалась библиотека jsonlines

По непонятной причине на Colab через пару дней после успешной загрузки датасета сайги код перестал выполняться с ошибкой, что не может сохранить кешированный датасет. ПРишлось перенести работу на Каггл.

Так же модель генерировала только один вариант ответа на несколько разных вопросов. Попытка штрафовать за повторения приводила к бесконечной итерации кода при выполнении и невозможности завершить его и выдать результат. странная галлюцинация, точную причину найти не смог, предполагаю, что это особенность сильно квантованной модели.



# Итог

Загрузил квантованную модель сайги 2k, протестировал генерацию ответов от нее. первая генерация проходит хорошо - за тем модель начинает галлюцинировать. Сравнил ответы сайги и GPT-3.5-turbo. Первый ответ оказался логичным и связанным, далее пошла галлюц